In [1]:
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset

device = "cuda"
path_model = "/home/chwenjun225/Projects/Foxer/models/DeepSeek-R1-Distill-Qwen-1.5B"
path_dataset = "/home/chwenjun225/Projects/Foxer/datasets/cot-2"

In [2]:
from unsloth import FastLanguageModel

max_seq_length = 2048 
dtype = None 
load_in_4bit = True # Training với False sau 

model, tokenizer = FastLanguageModel.from_pretrained(
	model_name = path_model,
	max_seq_length = max_seq_length,
	dtype = dtype,
	load_in_4bit = load_in_4bit,
	# token = hf_token, 
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-04 14:24:11 __init__.py:183] Automatically detected platform cuda.
==((====))==  Unsloth 2025.1.8: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
/home/chwenjun225/Projects/Foxer/models/DeepSeek-R1-Distill-Qwen-1.5B does not have a padding token! Will use pad_token = <|vision_pad|>.


In [3]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are an expert across multiple domains. Please apply a step-by-step Chain-of-Thought (CoT) approach to carefully analyze and solve the problem in a logical and accurate manner.

Based on the given question, apply the most relevant expertise and provide a structured response.

### Prompt:
{}

### Response:
<thinking>{}"""

In [4]:
prompt = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(prompt, "")], return_tensors="pt").to(device)

outputs = model.generate(
	input_ids=inputs.input_ids,
	attention_mask=inputs.attention_mask,
	max_new_tokens=1200,
	use_cache=True,
)

response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<thinking>Okay, so I have this question about a 61-year-old woman with some gynecological exam findings. Let me try to unpack this step by step. First, she's a 61-year-old, which suggests she's had her eyesight checked for a long time. The question is about what cystometry would reveal about her residual volume and detrusor contractions. 

So, the patient has a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night. That's interesting. I remember that during the night, especially when someone coughs or sneezes, there's a higher chance of leakage because the blood can drain through the nose, especially if the eyes are open. But when she's lying down or not coughing, the leakage doesn't happen. 

Now, she's undergoing a gynecological exam and a Q-tip test. Let me think about what that entails. The Q-tip test is used to assess the presence of detrusor contractions. Detrusor contractions are a type of contractions that cause gas, especia

In [5]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are an expert across multiple domains. Please apply a step-by-step Chain-of-Thought (CoT) approach to carefully analyze and solve the problem in a logical and accurate manner.

Based on the given question, apply the most relevant expertise and provide a structured response.

### Prompt:
{}

### Response:
{}
"""

In [6]:
from datasets import load_dataset
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
	prompts = examples["prompt"]
	responses = examples["response"]
	texts = []
	for prompt, response in zip(prompts, responses):
		text = train_prompt_style.format(prompt, response) + EOS_TOKEN
		texts.append(text)
	return {"texts": texts}

dataset = load_dataset(path_dataset, split="train[0:6000]", trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched=True)
dataset['texts'][0]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are an expert across multiple domains. Please apply a step-by-step Chain-of-Thought (CoT) approach to carefully analyze and solve the problem in a logical and accurate manner.\n\nBased on the given question, apply the most relevant expertise and provide a structured response.\n\n### Prompt:\nExplain the difference between truth and justification in epistemology. How can these concepts lead to different combinations of true/false beliefs and justified/unjustified beliefs? Provide examples for each possible combination.\n\n### Response:\n<thinking>\n<initial_analysis>\nTask: Explain the difference between truth and justification in epistemology and provide examples of

In [7]:
model = FastLanguageModel.get_peft_model(
	model,
	r=16,  
	target_modules=[
		"q_proj",
		"k_proj",
		"v_proj",
		"o_proj",
		"gate_proj",
		"up_proj",
		"down_proj",
	],
	lora_alpha=16,
	lora_dropout=0,  
	bias="none",  
	use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
	random_state=3407,
	use_rslora=False,  
	loftq_config=None,
)

Unsloth 2025.1.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
	model=model,
	tokenizer=tokenizer,
	train_dataset=dataset,
	dataset_text_field="texts",
	max_seq_length=max_seq_length,
	dataset_num_proc=2,
    packing=False, # Can make training 5x faster for short sequences.
	args=TrainingArguments(
		per_device_train_batch_size=2,
		gradient_accumulation_steps=4,
		# Use num_train_epochs = 1, warmup_ratio for full training runs!
		warmup_steps=5,
        max_grad_norm=0.3,
		max_steps=60,
		learning_rate=2e-4,
		fp16=not is_bfloat16_supported(),
		bf16=is_bfloat16_supported(),
		logging_steps=10,
		optim="adamw_8bit",
		weight_decay=0.01,
		lr_scheduler_type="linear",
		seed=3407,
		output_dir="./results",
        report_to="tensorboard",
	),
)

Map (num_proc=2):   0%|          | 0/6000 [00:00<?, ? examples/s]

In [9]:
# Nếu tiếp tục train sử dụng --> trainer_stats = trainer.train("checkpoint-9500") 
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
10,2.040000
20,1.559100
30,1.283300
40,1.186800
50,1.182300
60,1.113700


In [10]:
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"

FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
	input_ids=inputs.input_ids,
	attention_mask=inputs.attention_mask,
	max_new_tokens=1200,
	use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<thinking> <conscious_thought>
<initial_analysis>
Task: Identify the most likely outcome of a cystometry test based on gynecological exam and Q-tip results
Approach: Analyze the findings from gynecological exam and cystometry test
Complexity: Medium
</conscious_thought>
<conscious_thought>
Key points:
- Long history of involuntary urine loss
- No leakage at night
- Results from gynecological exam and Q-tip test
</conscious_thought>
<step_by_step>
1. Analyze gynecological exam results:
   - No leakage at night suggests normal intrauterine spaces
   - Urinary retention during activities (coughing, sneezing) is unusual
2. Consider Q-tip test results:
   - No contractions indicate no detrusor contractions
3. Apply cystometry results:
   - No contractions likely indicate no residual volume
   - No contractions suggest no detrusor contractions
</step_by_step>
<reflection>
This analysis aligns with the initial findings from the gynecological exam and Q-tip test. The results from the cystomet

In [11]:
new_model_local = "finetuned_DeepSeek-R1-Distill-Qwen-1.5B_finetune_CoT_ReAct"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 15.51 out of 31.25 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 100.54it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [ ]:
!lm_eval --model hf \
--model_args pretrained="/home/chwenjun225/Projects/Foxer/notebooks/DeepSeek-R1-Distill-Qwen-1.5B_finetune_CoT_ReAct/finetuned_DeepSeek-R1-Distill-Qwen-1.5B_finetune_CoT_ReAct" \
--tasks lambada_openai,hellaswag,piqa,arc_easy,arc_challenge,winogrande,openbookqa \
--device cuda \
--batch_size auto 